2021/06/10 -> 2023/04/29

- MeCab
- 形態素解析
- 頻度分析・単語 n-gram
  - n-gram 言語モデル
  - スムージング
  - 実装
- Word2Vec

参考
- 2020年度人工知能専門演習I [AI] (柴田先生)

# MeCab
形態素解析ツール・ライブラリ

## インストール

### Windowsの場合
`> pip install mecab-python3 unidic-lite`
- たぶんこれが最速
- ただし `-Ochasen` には非対応．
  - 後述の辞書の問題で，ipadicなどを使えばchasenも使える．
- [公式](https://github.com/SamuraiT/mecab-python3)によると https://learn.microsoft.com/en-US/cpp/windows/latest-supported-vc-redist の `vc_redist.x64.exe` も必要らしい？
- MeCab本体のインストール（推奨）
  - https://github.com/ikegami-yukino/mecab/releases の `MeCab-64-0.996.2.exe`
    - 文字コードはUTF-8推奨
  - `/Program Files/MeCab/bin/` をPathに追加する（インストール先を変更していなければ）．

### Colab, Linuxの場合
```sh
!apt install mecab-ipadic-utf8
!ln -s /etc/mecabrc /usr/local/etc/mecabrc
!pip install mecab-python3
```

### Macの場合
```sh
$ brew install mecab mecab-ipadic mecab-unidic
$ pip install mecab-python3
```
- 辞書の場所：`/usr/local/lib/mecab/dic/`
- 設定ファイル：`/usr/lobal/etc/mecabrc`

## 補足
MeCabは本来Pythonライブラリでなくただの形態素解析用CLIソフト。辞書とセットで使う。
- 環境により辞書のDLが必要。
- 辞書により使えるコマンドも違う。

辞書
- ipadic
  - MeCab標準
  - 2007年とかのもので古い．
  - `pip install ipadic` し，`import ipadic; MeCab.Tagger(ipadic.MECAB_ARGS)` で使える．
- unidic
  - chasenには対応していない．
- unidic-lite
  - 軽量版unidic
  - `pip install unidic-lite` し，そのまま使える．
- [mecab-ipadic-NEologd](https://github.com/neologd/mecab-ipadic-neologd)
  - 新語に強い．かつて最強とされていたが，更新は2020年で止まっている．
  - Windowsで使う場合，辞書をコンパイルするためにWSLを経由する必要がある．
    ```sh
    $ sudo apt install mecab mecab-ipadic-utf8 libmecab-dev make
    $ sudo git clone --depth 1 https://github.com/neologd/mecab-ipadic-neologd.git
    $ cd mecab-ipadic-neologd
    $ sudo ./bin/install-mecab-neologd -n
    ```
    - `Do you want to install mecab-ipadic-NEologd?` と訊かれたら `yes`．
    - `/usr/lib/x86_64-linux-gnu/mecab/dic/mecab-ipadic-neologd/` にコンパイルされた辞書が作成される．
    - ディレクトリごとWindowsの分かりやすいところにコピーする．
    - 使うときは `MeCab.Tagger("-d コピー先/mecab-ipadic-neologd)`
    - Windowsでも`bin/install-mecab-neologd`動く説？

Python用ラッパーライブラリ
- mecab-python3
  - 公式
- mecab
  - 有志がOSによらずインストールできるようにしたもの
- mecab-python-windows
  - 上記の有志がwindows用に作っていたもの。現在はmecabに統合され使えない。

# 形態素解析

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import MeCab

In [2]:
wakati = MeCab.Tagger("-Owakati")   # 分かち書き
chasen = MeCab.Tagger("-Ochasen")   # 分かち書き + 品詞分類
print(wakati.parse("すもももももももものうち")) # 'すもも も もも も もも の うち \n'
print(chasen.parse("すもももももももものうち")) # 表層形、読み、原型、品詞

すもも も もも も もも の うち 

すもも	スモモ	すもも	名詞-一般		
も	モ	も	助詞-係助詞		
もも	モモ	もも	名詞-一般		
も	モ	も	助詞-係助詞		
もも	モモ	もも	名詞-一般		
の	ノ	の	助詞-連体化		
うち	ウチ	うち	名詞-非自立-副詞可能		
EOS



In [3]:
with open("./data/neko.txt", encoding="utf8") as f:
    text = f.read()
morphemes = np.array([m[:4] for w in chasen.parse(text).split("\n") if len(m:=w.split())>=4])
words, poses = morphemes[:, 0], morphemes[:, 3]
print(morphemes.shape)  # 延べ語数は 205981
print(words[:10])       # 単語
print(poses[:10])       # 品詞

(205981, 4)
['一' '吾輩' 'は' '猫' 'で' 'ある' '。' '名前' 'は' 'まだ']
['名詞-数' '名詞-代名詞-一般' '助詞-係助詞' '名詞-一般' '助動詞' '助動詞' '記号-句点' '名詞-一般' '助詞-係助詞'
 '副詞-助詞類接続']


# 頻度分析・単語 n-gram
nltk: 自然言語処理ライブラリ

In [4]:
from nltk import ngrams, FreqDist
print(*FreqDist(words).most_common(5))  # 単語頻度順
print(*FreqDist(poses).most_common(5))  # 品詞頻度順
print(*FreqDist(words[poses=="名詞-一般"]).most_common(5))  # 一般名詞頻度順

('の', 9194) ('。', 7486) ('て', 6873) ('、', 6772) ('は', 6422)
('名詞-一般', 27470) ('動詞-自立', 24557) ('助詞-格助詞-一般', 20419) ('助動詞', 19810) ('助詞-接続助詞', 12067)
('主人', 932) ('人', 355) ('迷亭', 329) ('先生', 274) ('人間', 272)


単語 n-gram
- 「で」「ある」「。」は 3-gram (tri-gram) 

In [5]:
# uni-gram, bi-gram, tri-gram の頻度分布
for n in 1, 2, 3:
    fd = FreqDist(ngrams(words, n))
    print(fd.most_common(5), len(fd))

[(('の',), 9194), (('。',), 7486), (('て',), 6873), (('、',), 6772), (('は',), 6422)] 13589
[(('」', '「'), 1951), (('し', 'て'), 1233), (('て', 'いる'), 1105), (('」', 'と'), 1081), (('で', 'ある'), 960)] 73151
[(('で', 'ある', '。'), 759), (('て', 'いる', '。'), 482), (('し', 'て', 'いる'), 287), (('を', 'し', 'て'), 261), (('か', '」', '「'), 217)] 140130


## n-gram 言語モデル

- **定義）** 単語列 $w_{1..n-1}$ に $w_n$ が続く確率は $$P_n(w_n | w_{1..n-1}) := \frac{w_{1..n} の頻度}{w_{1..n-1} の頻度} $$
- **例）**「である」に「。」が続く確率は $$ P_3(。|である) = \frac{"である。" の頻度}{"である" の頻度} $$
- プログラム的には $$ models[3][("で", "ある")]["。"] = \frac{fds[3][("で", "ある", "。")]}{fds[2][("で", "ある"])} = \frac{759}{960} = 0.790625 $$

## スムージング

- **Add-α**
  - ちょっと補正（頻度に α を加算）した頻度を使う：$$ \widehat{P}_n(w_n | w_{1..n-1}) := \frac{w_{1..n} の頻度 + \alpha}{w_{1..n-1} の頻度 + \alpha K} $$
    - $K = len(fds[1])$：異なり語数
    - α：定数（1/Kとか）
- Good Turing
  - 経験カウントから、統計的に事後確率を推定する。
- **Back-off**
  - $P_3(。|である)=0$ の場合、代わりに $P_2(。|ある)$ をその値とする。
    - それも 0 なら $P_1(。)$ を使う。
- Add-α + Back-off $$ P_3(。|である) := d \widehat{P}_2(。|ある) $$
  - バックオフの係数 $d = \frac{\alpha (K - fds[2][("で", "ある")])}{fds[2][("で", "ある")] + \alpha K}$
- 補間
  - (1-λ)(n-gramでのカウントから計算される確率) + λ((n-1)-gramでの確率)
- Add-α + 補間 (interporation) $$ P_3(。|である) := \widehat{P}_3(。|である) + d P_2(。|ある) $$
  - 補間の係数 $d = \frac{\alpha K}{fds[2][("で", "ある")] + \alpha K}$
- **Modified Kneser-Ney**
  - 後で勉強する

## 実装

In [6]:
class NGram:
    def __init__(self, n, words):
        self.n = n
        # fds[k] は k-gram の頻度分布。0-gram の頻度は延べ語数とする
        fds = [{(): len(words)}] + [FreqDist(ngrams(words, k)) for k in range(1, n+1)]

        # models[k][(k-1)-gramのtuple] は (k-1)-gram に続く形態素の頻度分布
        self.models = {}
        for k in range(1, n+1):
            model = {prev: FreqDist() for prev in fds[k-1]}
            for kgram, freq in fds[k].items():
                model[kgram[:-1]][kgram[-1]] = freq / fds[k-1][kgram[:-1]]
            self.models[k] = model
        
    # 文生成
    def generate(self, start: str):
        sentense = wakati.parse(start).split()
        for _ in range(100):
            for k in range(self.n, 0, -1):
                # 未知語の次は uni-gram で最頻の「の」から続ける
                candidates = self.models[k].get(tuple(sentense[len(sentense)-k+1:]))
                if candidates:
                    sentense.append(candidates.max())
                    break
            if sentense[-1] == "。":
                break
        return "".join(sentense)

In [7]:
wv = NGram(6, words)
print(wv.generate("Artificial Intelligence"))
print(wv.generate("メロスは激怒した。"))
print(wv.generate("お前は今まで食ったパンの枚数を覚えているのか？"))

ArtificialIntelligenceのは、いかに卒業したての理学士にせよ、あまり能がなさ過ぎる。
メロスは激怒した。「御めえは今までに鼠を何匹とった事がある」智識は黒よりも余程発達しているつもりだが腕力と勇気とに至っては到底黒の比較にはならないと覚悟はしていたものの、この問に接したる時は、さすがに極りが善くはなかった。
お前は今まで食ったパンの枚数を覚えているのか？」「ええ、すると会社の男が、それは死ななければ無論保険会社はいりません。


In [8]:
print(wv.generate("「今日はいい天気ですね」「"))
print(wv.generate("「最低最悪だった」「"))
print(wv.generate("「あなたの名前は何ですか」「"))

「今日はいい天気ですね」「ジャムばかりじゃないんです、ほかに買わなけりゃ、ならない物もあります」と妻君は大に不平な気色を両頬に漲らす。
「最低最悪だった」「まさか」と細君が小さい声を出すと、「本当ですか」と寒月君が笑う。
「あなたの名前は何ですか」「あら御主人だって、妙なのね。


# Word2vec

- 単語をベクトルで表現する
- ベクトルは単語の埋め込み表現（意味みたいなもの）を考慮している
    - 類似する単語は類似するベクトルを持つ
    - 良いモデルでは $王 - 男 \fallingdotseq 女王 - 女$ とかも成り立つ
- 対義語に弱い
    - 逆ベクトルは「無関係」な言葉になる
    - 学習時に「学習データであるテキストにおける出現が近い単語」同士が近いベクトルになるため？
    - https://qiita.com/youwht/items/f21325ff62603e8664e6
- ライブラリ：gensim
    - [公式](https://radimrehurek.com/gensim/index.html)
    - [Gensim の Word2Vec を試す](https://qiita.com/propella/items/febc423998fd210800ca) (2021-06-22)

In [10]:
# 分かち書きから学習（分かち書きデータをファイル出力し，それをText8Corpusに投げる）
with open("./data/neko_wakati.txt", "w", encoding="utf8") as f:
    f.write(wakati.parse(text))

from gensim.models.word2vec import Word2Vec, Text8Corpus
wv = Word2Vec(Text8Corpus("./data/neko_wakati.txt")).wv
print(wv["猫"].shape)
print(wv.most_similar("人間"))
print(wv.most_similar("猫"))
print(wv.similarity("猫", "人間"))      # 類似度
print(wv.most_similar("猫", "人間"))    # 猫 - 人間

(100,)
[('猫', 0.9906332492828369), ('なる', 0.9789209961891174), ('自分', 0.9779163599014282), ('ぬ', 0.9746915698051453), ('この', 0.9734659194946289), ('する', 0.9732460379600525), ('人', 0.9717381000518799), ('彼', 0.9711594581604004), ('ため', 0.9711257219314575), ('我々', 0.9698050618171692)]
[('人間', 0.9906331896781921), ('自分', 0.9867601990699768), ('必要', 0.9851629137992859), ('者', 0.9847874641418457), ('する', 0.9846217036247253), ('にとって', 0.9823601841926575), ('ぬ', 0.9820329546928406), ('なる', 0.9816380143165588), ('思わ', 0.9759395718574524), ('碌', 0.9759304523468018)]
0.9906332
[('利き', 0.8413695096969604), ('よ', 0.8046698570251465), ('」', 0.8007706999778748), ('休み', 0.7891527414321899), ('釣れ', 0.785506010055542), ('ね', 0.7767887711524963), ('です', 0.76649409532547), ('そう', 0.7653157711029053), ('生きる', 0.7622565031051636), ('もん', 0.7542473077774048)]


学習済みモデル
- fastText
    - 2016年にFacebookが公開したNLPライブラリ
    - [fastTextの学習済みモデルを公開しました](https://qiita.com/Hironsan/items/513b9f93752ecee9e670)
- word2vec-google-news-300
    - gensim.downloader でDLできる

In [12]:
%%time
import gensim.downloader as api
wv = api.load("word2vec-google-news-300")

[==================================================] 100.0% 1662.8/1662.8MB downloaded
CPU times: total: 2min 57s
Wall time: 8min 54s


In [14]:
print(wv["cat"].shape)
print(wv.most_similar("dog"))
print(wv.most_similar("cat"))
print(wv.similarity("cat", "dog"))      # 類似度
print(wv.most_similar("cat", "dog"))    # 猫 - 人間

(300,)
[('dogs', 0.8680489659309387), ('puppy', 0.8106428384780884), ('pit_bull', 0.780396044254303), ('pooch', 0.7627376914024353), ('cat', 0.7609457969665527), ('golden_retriever', 0.7500901818275452), ('German_shepherd', 0.7465174198150635), ('Rottweiler', 0.7437615394592285), ('beagle', 0.7418621778488159), ('pup', 0.740691065788269)]
[('cats', 0.8099379539489746), ('dog', 0.760945737361908), ('kitten', 0.7464985251426697), ('feline', 0.7326234579086304), ('beagle', 0.7150582671165466), ('puppy', 0.7075453400611877), ('pup', 0.6934291124343872), ('pet', 0.6891531348228455), ('felines', 0.6755931973457336), ('chihuahua', 0.6709762215614319)]
0.76094574
[('Tia_Dalma_Naomie_Harris', 0.2577422261238098), ('Mouseland', 0.2575077712535858), ('antennal', 0.25083598494529724), ('chironomid', 0.24755403399467468), ('floo', 0.2461230456829071), ('fiendish_plot', 0.2458873838186264), ('architraves', 0.2403106689453125), ('mouse', 0.23750734329223633), ('flitting', 0.23703250288963318), ('pipi